In [12]:
import os
import pandas as pd
import json
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [72]:
LAGT = pd.read_parquet("../data/large_files/LAGT_v4-0.parquet")

In [73]:
LAGT

,author_id,doc_id,filename,author,title,sentences,lemmatized_sentences,source,lemmata_source,not_before,not_after,tlg_epithet,genre,provenience,wordcount,lemmatacount
0,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"[, , ., . . . . ., — πρός ἥν ( ,, ), ὁ Πινυτός...","[[], [], [], [], [πινυτός, ἀντιγράφω], [θαυμάζ...",glaux1,glaux1,101.0,200.0,[],None,christian,109,34
1,tlg0005,tlg0005.tlg003,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,[Οὐδενός εὐνάτειρα Μακροπτολέμοιο δέ μάτηρ μαί...,"[[εὐνητήρ, μακροπτολέμον, μήτηρ, μαῖα, ἀντιπέτ...",glaux1,glaux1,-400.0,-201.0,[Bucolici],None,pagan,95,59
2,tlg0006,tlg0006.tlg020,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,[ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσ...,"[[φημί, γῆ, ἐκλείπω, πόλις, ξενοῦσθαι], [πάτρα...",glaux1,glaux1,-500.0,-401.0,[Tragici],None,pagan,21516,10315
3,tlg0007,tlg0007.tlg146,tlg0007.tlg146.1st1K-grc1.xml,Plutarch,Παροιμίαι αἷς Ἀλεξανδρεῖς ἐχρῶντο,[Οἴκοι τά Μιλήσια: ἐπί τῶν ὅποι μή προςήκει τή...,"[[μιλήσιος, πργοςήκω, τρυφή, ἐπιδείκνυμι], [ἀρ...",glaux1,glaux1,1.0,200.0,"[Biographi, Philosophici/-ae]",None,pagan,3206,1496
4,tlg0007,tlg0007.tlg147,tlg0007.tlg147.1st1K-grc1.xml,Plutarch,Ἐκλογὴ περὶ τῶν ἀδυνάτων,"[Κατά πετρῶν σπείρεις., Πλίνθον πλύνεις., Δικτ...","[[πέτρα, σπείρω], [πλίνθος, πλύνω], [δίκτυον, ...",glaux1,glaux1,1.0,200.0,"[Biographi, Philosophici/-ae]",None,pagan,195,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959,tlg2062,tlg2062.tlg041,None,Joannes Chrysostomus,De sancto hieromartyre Babyla,"[Ἐγώ μέν ἐβουλόμην ἀποδοῦναι τό χρέος τήμερον,...","[[βούλομαι, ἀποδίδωμι, χρέος], [γίγνομαι, ὑπισ...",exprecce,grecy,301.0,500.0,None,"Encom., Hagiogr., Homilet.",christian,2468,1011
1960,tlg2062,tlg2062.tlg043,None,Joannes Chrysostomus,De sancta Pelagia virgine et martyre,"[Εὐλογητός ὁ Θεός·, καί γυναῖκες θανάτου λοιπό...","[[εὐλογητος, θεός], [γυνή, θάνατος, λοιπός, κα...",exprecce,grecy,301.0,500.0,None,"Encom., Hagiogr., Homilet.",christian,2884,1227
1961,tlg2062,tlg2062.tlg050,None,Joannes Chrysostomus,De sanctis martyribus,"[Χθές μαρτύρων ἡμέρα,, ἀλλά καί σήμερον μαρτύρ...","[[μάρτυς, ἡμέρα], [μάρτυς, ἡμέρα], [μάρτυς, ἡμ...",exprecce,grecy,301.0,500.0,None,"Encom., Hagiogr., Homilet.",christian,3586,1505
1962,tlg2062,tlg2062.tlg511,None,Joannes Chrysostomus,In sanctum Julianum martyrem,"[Εἰ ἐν τῇ γῇ τοιαῦται τοῖς μάρτυσιν αἱ τιμαί,,...","[[γῆ, τοιοῦτος, μάρτυς, τιμή], [ἀποδημία, πλέκ...",exprecce,grecy,301.0,500.0,None,"Encom., Hagiogr.",christian,4573,1893


In [74]:
LAGT[LAGT["doc_id"].isin(["tlg4163.tlg001", "tlg5451.tlg001", "tlg2679.tlg002"])]

,author_id,doc_id,filename,author,title,sentences,lemmatized_sentences,source,lemmata_source,not_before,not_after,tlg_epithet,genre,provenience,wordcount,lemmatacount
1874,tlg4163,tlg4163.tlg001,None,Telegonia,Telegonia,[ἤσθιεν ἁρπαλέως κρέα τ’ ἄσπετα καὶ μέθυ ἡδύ.],"[[ἐσθίω, κρέας, ἄσπετος, μέθυ, ἡδύς]]",glaux1,glaux1,NaN,NaN,None,None,None,9,5
1885,tlg2679,tlg2679.tlg002,None,,,"[εὐλογητὸς εἶ, κύριε ὁ θεός, ὁ δοὺς τῷ Σολομῶν...","[[εὐλογητός, εἰμί, κύριος, θεός, δίδωμι, Σολομ...",glaux1,glaux1,NaN,NaN,None,None,None,87,38
1943,tlg5451,tlg5451.tlg001,None,,Passio sancti Sabae Gothi (sub auctore Athanar...,"[Μαρτύριον τοῦ ἁγίου Σάβα τοῦ Γότθου., Ἡ ἐκκλη...","[[μαρτύριος, ἅγιος, σάβας, γότθης], [ἐκκλησία,...",exprecce,grecy,301.0,400.0,None,Hagiogr.,christian,1892,841


In [75]:
file_data = json.load(open(os.path.expanduser("../../ServiceAccountsKey.json")))
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(
    ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

lagt_metadata_gs = gc.open_by_url(
    "https://docs.google.com/spreadsheets/d/10pGulpiwzjUozVEVstKBwtftyDisSY9h7-kl82TVs0A/edit?usp=sharing")

In [76]:
data = lagt_metadata_gs.worksheet("additions_v1").get_all_records()
data[:10]

[{'doc_id': 'tlg0002.tlg001',
  'doc_id_comment': '',
  'corrected_doc_id': '',
  'author': 'Theognis',
  'title': 'Elegiae',
  'not_before': -600,
  'not_after': -501,
  'provenience': 'pagan',
  'Genre1': 'Eleg.',
  'Genre2': '',
  'Genre3': ''},
 {'doc_id': 'tlg0003.tlg002',
  'doc_id_comment': '',
  'corrected_doc_id': '',
  'author': 'Thucydides Atheniensis',
  'title': 'Epigramma',
  'not_before': -500,
  'not_after': -401,
  'provenience': 'pagan',
  'Genre1': 'Epigr.',
  'Genre2': '',
  'Genre3': ''},
 {'doc_id': 'tlg0004.tlg002',
  'doc_id_comment': '',
  'corrected_doc_id': '',
  'author': 'Diogenes Laertius',
  'title': 'Epigrammata (App. Anth.)',
  'not_before': 201,
  'not_after': 300,
  'provenience': 'pagan',
  'Genre1': 'Epigr.',
  'Genre2': '',
  'Genre3': ''},
 {'doc_id': 'tlg0005.tlg005',
  'doc_id_comment': '',
  'corrected_doc_id': '',
  'author': 'Theocritus',
  'title': 'Epigrammata',
  'not_before': -400,
  'not_after': -201,
  'provenience': 'pagan',
  'Genre1'

In [77]:
restructured_data = {}
for row in data:
    doc_id = row.pop("doc_id", None)  # Get the 'doc_id' and remove it from the record
    if doc_id:
        restructured_data[doc_id] = row 

In [78]:
restructured_data

{'tlg0002.tlg001': {'doc_id_comment': '',
  'corrected_doc_id': '',
  'author': 'Theognis',
  'title': 'Elegiae',
  'not_before': -600,
  'not_after': -501,
  'provenience': 'pagan',
  'Genre1': 'Eleg.',
  'Genre2': '',
  'Genre3': ''},
 'tlg0003.tlg002': {'doc_id_comment': '',
  'corrected_doc_id': '',
  'author': 'Thucydides Atheniensis',
  'title': 'Epigramma',
  'not_before': -500,
  'not_after': -401,
  'provenience': 'pagan',
  'Genre1': 'Epigr.',
  'Genre2': '',
  'Genre3': ''},
 'tlg0004.tlg002': {'doc_id_comment': '',
  'corrected_doc_id': '',
  'author': 'Diogenes Laertius',
  'title': 'Epigrammata (App. Anth.)',
  'not_before': 201,
  'not_after': 300,
  'provenience': 'pagan',
  'Genre1': 'Epigr.',
  'Genre2': '',
  'Genre3': ''},
 'tlg0005.tlg005': {'doc_id_comment': '',
  'corrected_doc_id': '',
  'author': 'Theocritus',
  'title': 'Epigrammata',
  'not_before': -400,
  'not_after': -201,
  'provenience': 'pagan',
  'Genre1': '',
  'Genre2': '',
  'Genre3': ''},
 'tlg0006

In [79]:
for doc_id, value_dict in restructured_data.items():
    # Find the corresponding row in LAGT DataFrame
    row_index = LAGT[LAGT['doc_id'] == doc_id].index
    if not row_index.empty:
        row_index = row_index[0]
        # Update values in LAGT DataFrame wherever the value in restructured_data is not empty
        for col, new_value in value_dict.items():
            if isinstance(new_value, str):
                new_value = new_value.replace("?", "")
            if new_value not in (None, ''):
                try: 
                    new_value = int(new_value)
                except:
                    pass
                if col == "corrected_doc_id":
                    col = "doc_id"
                LAGT.at[row_index, col] = new_value

In [80]:
LAGT[LAGT["doc_id"].isin(["tlg4163.tlg001", "tlg5451.tlg001", "tlg2679.tlg002"])]

,author_id,doc_id,filename,author,title,sentences,lemmatized_sentences,source,lemmata_source,not_before,not_after,tlg_epithet,genre,provenience,wordcount,lemmatacount,Genre1,Genre2,Genre3,doc_id_comment
1874,tlg4163,tlg4163.tlg001,None,Telegonia,Telegonia,[ἤσθιεν ἁρπαλέως κρέα τ’ ἄσπετα καὶ μέθυ ἡδύ.],"[[ἐσθίω, κρέας, ἄσπετος, μέθυ, ἡδύς]]",glaux1,glaux1,-600.0,-501.0,None,None,pagan,9,5,NaN,NaN,NaN,NaN
1885,tlg2679,tlg2679.tlg002,None,,Testamentum Salomonis (recensiones A et B) (ms...,"[εὐλογητὸς εἶ, κύριε ὁ θεός, ὁ δοὺς τῷ Σολομῶν...","[[εὐλογητός, εἰμί, κύριος, θεός, δίδωμι, Σολομ...",glaux1,glaux1,201.0,300.0,None,None,christian,87,38,NaN,NaN,NaN,NaN
1943,tlg5451,tlg5451.tlg001,None,,Passio sancti Sabae Gothi (sub auctore Athanar...,"[Μαρτύριον τοῦ ἁγίου Σάβα τοῦ Γότθου., Ἡ ἐκκλη...","[[μαρτύριος, ἅγιος, σάβας, γότθης], [ἐκκλησία,...",exprecce,grecy,301.0,400.0,None,Hagiogr.,christian,1892,841,NaN,NaN,NaN,NaN


In [81]:
def combine_genres(row):
    genres = [row['Genre1'], row['Genre2'], row['Genre3']]
    return [genre for genre in genres if pd.notna(genre)]

# Apply the function and update the 'genre' column
LAGT['genre'] = LAGT.apply(lambda row: combine_genres(row) if pd.isna(row['genre']) else [row['genre']], axis=1)

LAGT = LAGT.drop(columns=['Genre1', 'Genre2', 'Genre3', 'doc_id_comment'])


In [82]:
# "tlg0653.tlg002" should not exist anymore
# we should see ""tlg0653.tlg003" instead...
LAGT[LAGT["doc_id"].isin(["tlg4163.tlg001", "tlg5451.tlg001", "tlg2679.tlg002", "tlg0653.tlg003", "tlg0653.tlg002"])]

,author_id,doc_id,filename,author,title,sentences,lemmatized_sentences,source,lemmata_source,not_before,not_after,tlg_epithet,genre,provenience,wordcount,lemmatacount
1778,tlg0653,tlg0653.tlg003,None,Aratus Soleus,Epigrammata,"[αἰάζω Διότιμον, ὃς ἐν πέτραισι κάθηται, Γαργα...","[[αἰάζω, Διότιμος, πέτρα, κάθημαι, Γαργαρής, π...",glaux1,glaux1,-400.0,-201.0,None,[Epigr.],pagan,65,31
1874,tlg4163,tlg4163.tlg001,None,Telegonia,Telegonia,[ἤσθιεν ἁρπαλέως κρέα τ’ ἄσπετα καὶ μέθυ ἡδύ.],"[[ἐσθίω, κρέας, ἄσπετος, μέθυ, ἡδύς]]",glaux1,glaux1,-600.0,-501.0,None,[],pagan,9,5
1885,tlg2679,tlg2679.tlg002,None,,Testamentum Salomonis (recensiones A et B) (ms...,"[εὐλογητὸς εἶ, κύριε ὁ θεός, ὁ δοὺς τῷ Σολομῶν...","[[εὐλογητός, εἰμί, κύριος, θεός, δίδωμι, Σολομ...",glaux1,glaux1,201.0,300.0,None,[],christian,87,38
1943,tlg5451,tlg5451.tlg001,None,,Passio sancti Sabae Gothi (sub auctore Athanar...,"[Μαρτύριον τοῦ ἁγίου Σάβα τοῦ Γότθου., Ἡ ἐκκλη...","[[μαρτύριος, ἅγιος, σάβας, γότθης], [ἐκκλησία,...",exprecce,grecy,301.0,400.0,None,[Hagiogr.],christian,1892,841


In [85]:
LAGT.to_parquet("../data/large_files/LAGT_v4-1.parquet")

In [84]:
# save metadata for future usage
LAGT[['author_id', 'doc_id', 'filename', 'author', 'title', 'source', 'lemmata_source', 'not_before',
       'not_after', 'tlg_epithet', 'genre', 'provenience', 'wordcount',
       'lemmatacount']].to_csv("../data/LAGT_v4-1_metadata.csv", index=False)